In [2]:
import numpy as np
from time import time
from esutil.coords import sphdist
from multiprocessing import Pool
import pandas as pd
import tables
import sqlite3
import spherical_to_tangential as s2t
from scipy.optimize import curve_fit
import esutil

In [ ]:
def calcMedianAndResiduals2(table, ra0, dec0, ref_cri, gnomonic, debug):
    '''calculates median xi and eta along with residuals for a region of sky once'''
    '''table: pytables table that contains positions of PS1 objects'''
    #select rows that contain galaxies
    maskForGal = ref_cri#table.get_where_list(ref_cri)
    
    #total number of *detections* of galaxies
    noOfDetOfGal = sum(maskForGal)
    noOfGal = np.unique(table['obj_id'][maskForGal]).size
    #print "noOfGal:", noOfGal
    #print "noOfDetOfGal:", noOfDetOfGal
    #store the median ra and dec values for each galaxy 
    medianRaArray  = np.zeros(noOfGal)
    medianDecArray = np.zeros(noOfGal)

    ##################### updated by HJT#################
    medianResidualRaArray  = np.zeros(noOfGal)
    medianResidualDecArray = np.zeros(noOfGal) 
    #####################################################

    #store the residual values for each galaxy and for each epoch
    residualRaArray  = np.zeros(noOfDetOfGal)
    residualDecArray = np.zeros(noOfDetOfGal)
    
    #store the number of observations for each galaxy 
    noOfObsArray = np.zeros(noOfGal, dtype='u1')
    
    #store the objIDs of each galaxy -- we run it in the loop just for debugging purposes 
    #needs 64-bit unsigned integer datatype
    objIDarray = np.zeros(noOfGal, dtype='i8')
    
    # pluck out objIDs, RAs, DECs, and MJDs for each detection of galaxy
    # (works *much* faster than accessing individual pytable rows)
    objIDs  = table['obj_id'][maskForGal]
    #print "len(objIDs)", len(objIDs)
    #print "objIDs.size", objIDs.size
    raObjs  = table['ra'][maskForGal]
    decObjs = table['dec'][maskForGal]
    #########################(RA, DEC) ==> (xi, eta)#################################
    if(gnomonic):
	raObjs, decObjs, status = s2t.ds2tp(raObjs, decObjs, ra0, dec0) # transform the (RA, DEC)/degree into (xi, eta)/degree, but here (xi, eta) denoted by (raObjs, decObjs)
	#print "Number points in bad transformation:", np.sum(status)
    #################################################################################
    mjdObjs = table['mjd'][maskForGal]

    ##################### updated by HJT#################
    raErrObjs  = table['raErr'][maskForGal]  
    decErrObjs = table['decErr'][maskForGal] 
    #####################################################

    try:
        # store the first object values in here
        currObjID = objIDs[0]
        currRaObj  = [raObjs[0]]
        currDecObj = [decObjs[0]]
	##################### updated by HJT#################
    	currRaErrObj  = [raErrObjs[0]]
        currDecErrObj = [decErrObjs[0]] 
    	#####################################################
        currMjdObj = [mjdObjs[0]]
        noOfObsArray[0] = 1

        #set variables
        objIDcounter = 0
        pos = 0
        t1 = time()

        #it should start from the second row, so add one.
        for i in np.arange(noOfDetOfGal-1)+1:
            if debug: print 'on the row number',i
            objID = objIDs[i]
            ra    = raObjs[i]
            dec   = decObjs[i]
            mjd   = mjdObjs[i]
    	    ##################### updated by HJT#################
    	    raErr  = raErrObjs[i]  
    	    decErr = decErrObjs[i] 
    	    #####################################################

            if debug: print 'object ID',objID
            if debug: print 'ra of object',ra
            if debug: print 'dec of object',dec
            if debug: print 'mjd of object',mjd
            if (objID == currObjID):
                currRaObj.append(ra)
                currDecObj.append(dec)
                currMjdObj.append(mjd)
    	        ##################### updated by HJT#################
    	        currRaErrObj.append(raErr)
                currDecErrObj.append(decErr) 
    	        #####################################################

                if debug: print 'same object now'
                noOfObsArray[objIDcounter] += 1
            else:
                if debug: print 'different object now'
                if debug: print 'run functions for the', objIDcounter, 'object'
		currMjdSorted  = np.sort(currMjdObj)
    		deltaT = currMjdSorted[0:-1] - currMjdSorted[1:]
    		tBreakAt   = np.where(deltaT < (-100.0))[0]
    		currMjdBreakAt = currMjdSorted[tBreakAt+1]
		if debug: print "currMjdObj:", currMjdObj
                if(len(currMjdBreakAt) >= 3.0):#(noOfObsArray[objIDcounter] >= 2.0):
		    #print "tian test1,currMjdBreakAt", currMjdBreakAt
                    if debug: print 'no of obs ', noOfObsArray[objIDcounter]
                    # convert list into array
                    currRaObj = np.array(currRaObj)
                    currDecObj = np.array(currDecObj)
		    ##################### updated by HJT#################
		    currMjdObj = np.array(currMjdObj)
    	            currRaErrObj = np.array(currRaErrObj)
                    currDecErrObj = np.array(currDecErrObj)
		    '''currRaObj = currRaObj[currMjdObj<=currMjdBreakAt[2]]  ## Here notice we only choose the first 4 epochs 
		    currDecObj = currDecObj[currMjdObj<=currMjdBreakAt[2]]
		    currRaErrObj = currRaErrObj[currMjdObj<=currMjdBreakAt[2]]
		    currDecErrObj = currDecErrObj[currMjdObj<=currMjdBreakAt[2]]
		    currMjdObj = currMjdObj[currMjdObj<=currMjdBreakAt[2]]'''
    	            #####################################################

                    if debug: print 'current objID ra dec mjd :',currObjID, currRaObj, currDecObj, currMjdObj
                    # calculate and store medians
                    #medianRa = np.median(currRaObj)
                    #medianDec = np.median(currDecObj)
		    ##################### updated by HJT#################
    	            medianRa = np.sum(currRaObj/currRaErrObj**2)/np.sum(1/currRaErrObj**2)
                    medianDec = np.sum(currDecObj/currDecErrObj**2)/np.sum(1/currDecErrObj**2)
    	            #####################################################

                    if debug: print 'medianRa is', medianRa
                    if debug: print 'medianDec is', medianDec
                    # store median value for each object
                    medianRaArray[objIDcounter] = medianRa
                    medianDecArray[objIDcounter] = medianDec
                    objIDarray[objIDcounter] = currObjID
                    # calculate residual
                    residualRa = currRaObj - medianRa
                    residualDec = currDecObj - medianDec
                    if debug: print 'residualRa is', residualRa
                    if debug: print 'residualDec is', residualDec
                    if debug: print 'pos is', pos
                    if debug: print 'i is ',i
                    # store residuals in separate arrays
		    indTmp = np.arange(pos,i)
		    #indTmp = indTmp[currMjdObj<=currMjdBreakAt[2]]
                    residualRaArray[indTmp] = residualRa
                    residualDecArray[indTmp] = residualDec
		    ##################### updated by HJT#################
		    # store median value for each object
                    medianResidualRaArray[objIDcounter] = np.median(residualRa)
                    medianResidualDecArray[objIDcounter] = np.median(residualDec)
		    #####################################################

                # store the objID no matter what
                objIDarray[objIDcounter] = currObjID
                objIDcounter += 1
                currObjID = objID
                pos = i
                # start new ra, dec, and mjd arrays and
                # don't forget to increment the noOfObsArray for this object
                currRaObj = [raObjs[i]]
                currDecObj = [decObjs[i]]
		##################### updated by HJT#################
    	        currRaErrObj = [raErrObjs[i]]
                currDecErrObj = [decErrObjs[i]]
    	        #####################################################
                currMjdObj = [mjdObjs[i]]
                noOfObsArray[objIDcounter] += 1

        # after going through all of the rows, make sure to process the last
        # object, if it has enough observations
        i +=1

        print 'processing the last object now'
        currMjdSorted  = np.sort(currMjdObj)
    	deltaT = currMjdSorted[0:-1] - currMjdSorted[1:]
    	tBreakAt   = np.where(deltaT < (-100.0))[0]
    	currMjdBreakAt = currMjdSorted[tBreakAt+1]
        if(len(currMjdBreakAt) >= 3.0):#(noOfObsArray[objIDcounter] >= 2.0):
            if debug: print 'obs greater than three, adding the last object too'
            currRaObj = np.array(currRaObj)
            currDecObj = np.array(currDecObj)
	    ##################### updated by HJT#################
    	    currRaErrObj = np.array(currRaErrObj)
            currDecErrObj = np.array(currDecErrObj)
	    currMjdObj = np.array(currMjdObj)
	    '''currRaObj = currRaObj[currMjdObj<=currMjdBreakAt[2]]  ## Here notice we only choose the first 4 epochs 
	    currDecObj = currDecObj[currMjdObj<=currMjdBreakAt[2]]
	    currRaErrObj = currRaErrObj[currMjdObj<=currMjdBreakAt[2]]
	    currDecErrObj = currDecErrObj[currMjdObj<=currMjdBreakAt[2]]
	    currMjdObj = currMjdObj[currMjdObj<=currMjdBreakAt[2]]'''
    	    #####################################################
            # calculate and store medians
            #medianRa = np.median(currRaObj)
            #medianDec = np.median(currDecObj)

	    ##################### updated by HJT#################
    	    medianRa = np.sum(currRaObj/currRaErrObj**2)/np.sum(1/currRaErrObj**2)
            medianDec = np.sum(currDecObj/currDecErrObj**2)/np.sum(1/currDecErrObj**2)
    	    #####################################################
            if debug: print 'medianRa is', medianRa
            if debug: print 'medianDec is', medianDec
            medianRaArray[objIDcounter] = medianRa
            medianDecArray[objIDcounter] = medianDec
            # calculate residual
            residualRa = currRaObj - medianRa
            residualDec = currDecObj - medianDec
            if debug: print 'residualRa is', residualRa
            if debug: print 'residualDec is', residualDec
            if debug: print 'pos is', pos
            if debug: print 'i is ',i
            indTmp = np.arange(pos,i)
	    #indTmp = indTmp[currMjdObj<=currMjdBreakAt[2]]
            residualRaArray[indTmp] = residualRa
            residualDecArray[indTmp] = residualDec
	    ##################### updated by HJT#################
	    # store median value for each object
            medianResidualRaArray[objIDcounter] = np.median(residualRa)
            medianResidualDecArray[objIDcounter] = np.median(residualDec)
	    #####################################################
            # also need to store the objID, no matter what
            objIDarray[objIDcounter] = currObjID
       
        print 'total time taken is', time() - t1


    except ValueError:
        print "currObjID medianRa medianDec residualRa residualDec, pos, i, objIDcounter", currObjID, medianRa, medianDec, residualRa, residualDec, pos, i, objIDcounter
        pass
        
    return objIDarray, medianRaArray, medianDecArray, noOfObsArray, residualRaArray, residualDecArray, medianResidualRaArray, medianResidualDecArray

In [ ]:
def pixelTasks2(parameterListForPixel):
    """Workers will execute this function."""
    #unpack the input parameter list (i.e., separate them into different arrays)
    #objID -- contains the good galaxy IDs -- unique no of times
    #objIDs -- contains all the galaxy IDs corresponding to all the detections of galaxies
    #medRa and medDec -- contain good median values 
    #resRa and resDec -- contain the residual values for each detection of galaxy-- not good
    #ra/decAll -- ra/dec values for all the galaxies -- not good
    #pixAll -- pixel indices for all the galaxies -- not good
    pixelNo, pixRa, pixDec, objID, medRa, medDec, objIDs, MJDs, resRa, resDec, raAll, decAll, pixAll,mjdBreakAt, gnomonic, ra0, dec0 = parameterListForPixel


    #find objects within the searchRadius
    #angSepMask = sphdist(pixRa, pixDec, medRa, medDec) <= (searchRadius/60.0)
    #sphdist()求单位球面上两点之间的弧长
    distTmp = sphdist(pixRa, pixDec, medRa, medDec)
    angSepMask = (distTmp) <= (60.0/60.0)
    if(sum(angSepMask)>600):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:NGAL]
    elif(sum(angSepMask)<100):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:100]

    angSepMask = np.argsort(distTmp)
    angSepMask = angSepMask[0:NGAL]

    objInRadius = objID[angSepMask]
    #print "galaxy number in the circle:",len(objInRadius) 
    #print "min(decAll), max(decAll):", min(decAll), max(decAll)

    '''we use PANDAS'''
    try:
        flag1 = 1 
	r1 = time()
	temp = pd.Index(objIDs)
	inde = temp.get_indexer_for(objInRadius)
	#print 'Test1:',max(inde), len(inde)
	inde = inde[inde > -1]
	#print 'Test2:',len(inde)
	inde = np.array(inde, dtype='i8')
	#print 'Test3:',resRa
	#print 'Test3:',len(resRa), max(inde)
	resRaValuesInRadius = resRa[inde]
	#print 'Test4:',resRaValuesInRadius
	resDecValuesInRadius = resDec[inde]
	mjdInRadius = MJDs[inde]
	#print 'galaxy mjdInRadius:',mjdInRadius
	    
    except IndexError:
	#print 'galaxy mjdInRadius:',mjdInRadius
        return pixelNo, flag1, inde.size
	    
    #for storing median values for each epoch
    medianResidualRaEpochWise  = np.zeros(len(mjdBreakAt)+1)
    medianResidualDecEpochWise = np.zeros(len(mjdBreakAt)+1)
    indexInPixel = pixAll == pixelNo
    objIDinPixel = objID[indexInPixel]
   
    try :
	    flag2 = 2 
	    temp2 = pd.Index(objIDs)
	    inde2 = temp2.get_indexer_for(objIDinPixel)
	    inde2 = inde2[inde2 > -1]
	    inde2 = np.array(inde2, dtype='i8')
	    raInPixel = raAll[inde2]
	    decInPixel = decAll[inde2]
	    #########################(RA, DEC) ==> (xi, eta)#################################
	    if(gnomonic):
	        raInPixel, decInPixel, status = s2t.ds2tp(raInPixel, decInPixel, ra0, dec0)# transform the (RA, DEC)/degree into (xi, eta)/degree
	        #print "Number points in bad transformation:", np.sum(status)
	    #################################################################################
	    objInPixel = objIDs[inde2]
	    mjdInPixel = MJDs[inde2]
	    #print "time taken for pandas indexer", time() - r1
    except IndexError:
	    return pixelNo, flag2, inde2.size
    
    #if there are atleast three epochs present
    if (len(mjdBreakAt)>=3.0):
	for mjdIdx in range(0, len(mjdBreakAt)+1):
	    if(mjdIdx==0):
	        var = mjdBreakAt[mjdIdx]
		mjdIndexInPixel = mjdInPixel < var
		mjdIndex = mjdInRadius < var
	    elif(mjdIdx==len(mjdBreakAt)):
	        var = mjdBreakAt[mjdIdx-1]
		mjdIndexInPixel = mjdInPixel >= var
		mjdIndex = mjdInRadius >= var
	    else:
	        var = mjdBreakAt[mjdIdx]
		mjdIndex = (mjdInRadius >= mjdBreakAt[mjdIdx-1]) & (mjdInRadius < var)
                mjdIndexInPixel = (mjdInPixel >= mjdBreakAt[mjdIdx-1]) & (mjdInPixel < var)

            if(any(mjdIndex)):
                resRaValues  = resRaValuesInRadius[mjdIndex]
                resDecValues = resDecValuesInRadius[mjdIndex]
		#print "number in resRaValues:", len(resRaValues)
                assert(resRaValues.size > 0.0), "resRaValues is empty"
                assert(resDecValues.size > 0.0), "resDecValues is empty"
                medianResidualRaEpochWise[mjdIdx]  = np.median(resRaValues)
                medianResidualDecEpochWise[mjdIdx] = np.median(resDecValues)
		#print "np.median(resRaValues):", np.median(resRaValues)
                raInPixel[mjdIndexInPixel] -=  medianResidualRaEpochWise[mjdIdx]
                decInPixel[mjdIndexInPixel] -=  medianResidualDecEpochWise[mjdIdx]

    #calculate final ra/dec as the median of newRa/newDec values
    finalRaArray  = np.zeros(objIDinPixel.size)
    finalDecArray = np.zeros(objIDinPixel.size)
    medianRaError  = np.zeros(objIDinPixel.size)
    medianDecError = np.zeros(objIDinPixel.size)
    #print "galaxy number in the pixel:", len(finalRaArray)
    try:
        #take the current values separately
        obj   = objInPixel[0]
        raObj = [raInPixel[0]]
        decObj = [decInPixel[0]]
        mjdObj = [mjdInPixel[0]]
        #set variables
        counter = 0
        pos2 = 0
        t2   = time()
        for i in np.arange(objInPixel.size-1)+1:#np.arange(10)+1: #
            #print 'on the row number',i
            objID = objInPixel[i]
            ra  = raInPixel[i]
            dec = decInPixel[i]
            if(objID == obj):
                raObj.append(ra)
                decObj.append(dec)
            else:
                raObj = np.array(raObj)
                decObj= np.array(decObj)

                finalRaArray[counter]  = np.median(raObj)
                finalDecArray[counter] = np.median(decObj)
                #calculate rms values for ra/dec
                rmsRa  = 0.741*(np.percentile(raObj, 75) - np.percentile(raObj, 25))
                rmsDec = 0.741*(np.percentile(decObj, 75) - np.percentile(decObj, 25))
                #calculate uncertainity in median coordinates
                medianRaError[counter]  = np.sqrt((np.pi/2)/(raObj.size-1))*rmsRa
                medianDecError[counter] = np.sqrt((np.pi/2)/(decObj.size-1))*rmsDec
                obj  = objID
                pos2 = i
                counter +=1
                #make them lists again
                raObj  =[raInPixel[i]]
                decObj =[decInPixel[i]]
        #processing the last object now
        raObj  = np.array(raObj)
        decObj = np.array(decObj)
        finalRaArray[counter]  = np.median(raObj)
        finalDecArray[counter] = np.median(decObj)
        #calculate rms values for ra/dec
        rmsRa  = 0.741*(np.percentile(raObj, 75) - np.percentile(raObj, 25))
        rmsDec = 0.741*(np.percentile(decObj, 75) - np.percentile(decObj, 25))
        medianRaError[counter]  = np.sqrt((np.pi/2)/(raObj.size-1))*rmsRa
        medianDecError[counter] = np.sqrt((np.pi/2)/(decObj.size-1))*rmsDec
	#########################(xi, eta) ==> (RA, DEC)#################################
	if(gnomonic): # transform the (xi, eta)/radians into (RA, DEC)/degree
	    finalRaArray, finalDecArray = s2t.dtp2s(np.radians(finalRaArray), np.radians(finalDecArray), ra0, dec0)
	#################################################################################
        return objIDinPixel, finalRaArray, finalDecArray, medianRaError, medianDecError

    except IndexError:
        #pass
        print "index error"
        print pixelNo
        return np.array(objIDinPixel), np.array(finalRaArray), np.array(finalDecArray), np.array(medianRaError), np.array(medianDecError)

In [ ]:
def referenceGals(parameterListForPixel, dbname):
    db = sqlite3.connect(dbname)
    cursor = db.cursor()
    cursor.execute('''DROP TABLE IF EXISTS finalData''')
    db.commit()  
    cursor = db.cursor()
    cursor.execute( '''CREATE TABLE finalData(objID INT NOT NULL, ra REAL NOT NULL, dec REAL NOT NULL, raError REAL NOT NULL, decError REAL NOT NULL)''')
    db.commit() 
    #start workers
    pool = Pool(processes=15)
    ti = time()
    #chunk size is used to submit jobs in batches which reduces overhead
    iterator = pool.imap_unordered(pixelTasks2, parameterListForPixel[0:], chunksize=300)
    counter = 0
    noOfPixelsIterated = 0
    dat  = []
    for res in iterator:
        if (len(res)==3):
		print "there was an error with pandas :( ", res
	else:
		objID_, ra_, dec_, rErr_, dErr_ = res
		noOfPixelsIterated += 1
		print "objID in the pixel:",objID_
		#print "noOfPixelsIterated",noOfPixelsIterated
        #so that you donot encounter empty pixels! 
		if (objID_.size != 0):
			tmp = [(int(objID),float(ra), float(dec), float(rErr), float(dErr)) for objID, ra, dec, rErr, dErr in \
			    zip(objID_, ra_, dec_, rErr_, dErr_)]
			dat = dat + tmp
			counter = counter + len(objID_)
			'''if ((noOfPixelsIterated>50000)&(noOfPixelsIterated>int(noOfPixelsIterated/50000)*50000)): #
				cursor.executemany('INSERT INTO finalData (objID, ra, dec, raError, decError) VALUES (?, ?, ?, ?, ?)', tmp)
				db.commit()
				#print "committing last" '''
			if (noOfPixelsIterated % 5000 == 0):
				cursor.executemany('INSERT INTO finalData (objID, ra, dec, raError, decError) VALUES (?, ?, ?, ?, ?)', dat)
				db.commit()
				print "committing in 50000*N"
				dat = []
		elif (objID_.size ==0):
			print "zero array"   
		#print "counter", counter
    cursor.executemany('INSERT INTO finalData (objID, ra, dec, raError, decError) VALUES (?, ?, ?, ?, ?)', dat)
    db.commit()
    print "counter=total no of rows added to database", counter
    #terminate the pool of multiprocessors
    pool.terminate()
    # close connection to db
    db.close()

In [ ]:
def pixelTasksCombinedData(parameterList):
    #pixelNo is one number
    #pixelRa and dec are the ra/dec values of the pixel in consideration
    #galID are the object IDs of galaxies -- unique -- from the database
    #ra/decFinalGal are final ra/dec values of galaxies obtained from the database
    #galIDs -- galaxy IDs corresponding to all detections of galaxies
    #ra/decGal -- original ra and dec values of galaxies
    #starMjds --  mjd values corresponding to all the detections of stars
    pixelNo, pixelRa, pixelDec, galIDfinal, raFinalGal, decFinalGal, galIDs, raGal, decGal, galMjd , starIDs, starMjds, raStar, decStar, raErrStar, decErrStar, rMagStar, nObsStar, pixAllStar, mjdBreakAt, gnomonic, ra0, dec0 = parameterList

    distTmp = sphdist(pixelRa, pixelDec, raFinalGal, decFinalGal)
    '''angSepMask = (distTmp) <= (60.0/60.0)
    if(sum(angSepMask)>600):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:NGAL]
    elif(sum(angSepMask)<100):
        angSepMask = np.argsort(distTmp)
        angSepMask = angSepMask[0:100]'''

    angSepMask = np.argsort(distTmp)
    angSepMask = angSepMask[0:NGAL]

    #select unique galaxy ids within searchRadius
    uniqueGalIDinRadius = galIDfinal[angSepMask]
    raFinalGalInRadius  = raFinalGal[angSepMask]
    decFinalGalInRadius = decFinalGal[angSepMask]
    #########################(RA, DEC) ==> (xi, eta)#################################
    if(gnomonic):
	raFinalGalInRadius, decFinalGalInRadius, status = s2t.ds2tp(raFinalGalInRadius, decFinalGalInRadius, ra0, dec0)# transform the (RA, DEC)/degree into (xi, eta)/degree
	#print "Number points in bad transformation:", np.sum(status)
    #################################################################################

    try:
        flag1 = 1
        r1 = time()
        temp = pd.Index(galIDs)
        inde = temp.get_indexer_for(uniqueGalIDinRadius) # all the detections of gal
        inde = inde[inde > -1]
        inde = np.array(inde, dtype = 'i8')
        raGalInRadius  = raGal[inde]
        decGalInRadius = decGal[inde]
	#########################(RA, DEC) ==> (xi, eta)#################################
	if(gnomonic):
	    print "raGalInRadius, decGalInRadius:", raGalInRadius, decGalInRadius
	    raGalInRadius, decGalInRadius, status = s2t.ds2tp(raGalInRadius, decGalInRadius, ra0, dec0)# transform the (RA, DEC)/degree into (xi, eta)/degree
	    #print "Number points in bad transformation:", np.sum(status)
	#################################################################################
        galIDinRadius  = galIDs[inde]
        galMjdInRadius = galMjd[inde]

    except IndexError:
        return pixelNo, inde.size
         
    currentGalID  = galIDinRadius[0]
    currentRaGal  = [raGalInRadius[0]]
    currentDecGal = [decGalInRadius[0]]
    print "galIDinRadius:", galIDinRadius
    
    galCounter = 0 
    position = 0 
    noOfDetectionsOfGal = galIDinRadius.size
    #print "noOfDetectionsOfGal",noOfDetectionsOfGal
    
    offsetRaArray  = np.zeros(noOfDetectionsOfGal)
    offsetDecArray = np.zeros(noOfDetectionsOfGal)
    
    #run over all the objects within the searchRadius  
    t =  time()
    for k in np.arange(noOfDetectionsOfGal-1)+1:
        #print "on galaxy no", k 
        ID  = galIDinRadius[k]
        ra  = raGalInRadius[k]
        dec = decGalInRadius[k]
	
        if (ID == currentGalID):
                currentRaGal.append(ra)
                currentDecGal.append(dec)
                #print "same object now"
        else:
                #make them numpy arrays
		#print "currentGalID, uniqueGalIDinRadius:", currentGalID, uniqueGalIDinRadius[galCounter]
                currentRaGal  = np.array(currentRaGal)
                currentDecGal = np.array(currentDecGal)
		#print currentRaGal, raFinalGalInRadius[uniqueGalIDinRadius==currentGalID]
                offsetRa  = currentRaGal - raFinalGalInRadius[uniqueGalIDinRadius==currentGalID][0]
                offsetDec = currentDecGal - decFinalGalInRadius[uniqueGalIDinRadius==currentGalID][0]
                offsetRaArray[position:k]  = offsetRa
                offsetDecArray[position:k] = offsetDec
		#print "offsetRa, offsetDec", offsetRa, offsetDec
                #update counters and variables
	        galCounter+= 1
	        position   = k
	        currentGalID  = ID
	        currentRaGal  = [raGalInRadius[k]]
	        currentDecGal = [decGalInRadius[k]]
    #print "time taken to calculate offsets of galaxies from their final ra/dec", time()-t
    print "galCounter after loop", galCounter	

	
    # select stars inside the pixel
    resIndexInPixel =  pixAllStar == pixelNo
    print "pixelNo:", pixelNo, pixAllStar
    print "sum of star number in Pixel:", np.sum(resIndexInPixel)
    #uniqueStarIDinPixel = starID[resIndexInPixel]
    #match all detections of stars with those in the pixel
    #indexInPixel = np.in1d(starIDs, uniqueStarIDinPixel)
    #obtain original ra and dec values for stars within pixel
    raStarInPixel  = raStar[resIndexInPixel]
    decStarInPixel = decStar[resIndexInPixel]
    #########################(RA, DEC) ==> (xi, eta)#################################
    if(gnomonic):
	#print "raStarInPixel, decStarInPixel:", raStarInPixel, decStarInPixel
	raStarInPixel, decStarInPixel, status = s2t.ds2tp(raStarInPixel, decStarInPixel, ra0, dec0)# transform the (RA, DEC)/degree into (xi, eta)/degree
	#print "Number points in bad transformation:", np.sum(status)
    #################################################################################
    raErrStarInPixel  = raErrStar[resIndexInPixel]
    decErrStarInPixel = decErrStar[resIndexInPixel]
    rMagStarInPixel = rMagStar[resIndexInPixel]
    nObsStarInPixel = nObsStar[resIndexInPixel]
    #raStarErrInPixel  = np.zeros(np.sum(resIndexInPixel))
    #decStarErrInPixel = np.zeros(np.sum(resIndexInPixel))
    starIDInPixel  = starIDs[resIndexInPixel]
    mjdStarInPixel = starMjds[resIndexInPixel]
    print 'sum number of resIndexInPixel', np.sum(resIndexInPixel)

    medianRaOffsetEpochWise = np.zeros(len(mjdBreakAt)+1)
    medianDecOffsetEpochWise = np.zeros(len(mjdBreakAt)+1)
    #avgMjd = np.zeros(len(mjdBreakAt)+1)

    if (len(mjdBreakAt)>= 1.0):
        for mjdIdx in range(0, len(mjdBreakAt)+1):
	    if(mjdIdx==0):
	        var = mjdBreakAt[mjdIdx]
		mjdIndexInPixel = mjdStarInPixel < var
		mjdIndex = galMjdInRadius < var
	    elif(mjdIdx==len(mjdBreakAt)):
	        var = mjdBreakAt[mjdIdx-1]
		mjdIndexInPixel = mjdStarInPixel >= var
		mjdIndex = galMjdInRadius >= var
	    else:
	        var = mjdBreakAt[mjdIdx]
		mjdIndex = (galMjdInRadius >= mjdBreakAt[mjdIdx-1]) & (galMjdInRadius < var)
                mjdIndexInPixel = (mjdStarInPixel >= mjdBreakAt[mjdIdx-1]) & (mjdStarInPixel < var)
            if (any(mjdIndex)):
                offsetRaValues  = offsetRaArray[mjdIndex]
                offsetDecValues = offsetDecArray[mjdIndex]
                medianRaOffsetEpochWise[mjdIdx]  = np.median(offsetRaValues)
                medianDecOffsetEpochWise[mjdIdx] = np.median(offsetDecValues)
                #update the ra/dec - replace old values
                raStarInPixel[mjdIndexInPixel] -= medianRaOffsetEpochWise[mjdIdx]
                decStarInPixel[mjdIndexInPixel]-= medianDecOffsetEpochWise[mjdIdx]
		#print "np.median(offsetRaValues):", np.median(offsetRaValues)
		if((offsetRaValues.size>1)):
		    raErrTmp = np.sqrt((np.pi/2)/(offsetRaValues.size-1)) * \
		        (0.741*(np.percentile(offsetRaValues, 75) - np.percentile(offsetRaValues, 25)))
		    decErrTmp = np.sqrt((np.pi/2)/(offsetDecValues.size-1)) * \
		        (0.741*(np.percentile(offsetDecValues, 75) - np.percentile(offsetDecValues, 25)))
		    raErrStarInPixel[mjdIndexInPixel] = np.sqrt(raErrStarInPixel[mjdIndexInPixel]**2 + raErrTmp**2) # 
		    decErrStarInPixel[mjdIndexInPixel] = np.sqrt(decErrStarInPixel[mjdIndexInPixel]**2 + decErrTmp**2) # 
                #avgMjd[var1] = (mjdStarInPixel[mjdIndexInPixel].max() - mjdStarInPixel[mjdIndexInPixel].min() )/2

    finalObjIDstar    = starIDInPixel
    finalRaArrayStar  = raStarInPixel
    finalDecArrayStar = decStarInPixel

    #########################(xi, eta) ==> (RA, DEC)#################################
    if(gnomonic): # transform the (xi, eta)/radians into (RA, DEC)/degree
	finalRaArrayStar, finalDecArrayStar = s2t.dtp2s(np.radians(finalRaArrayStar), np.radians(finalDecArrayStar), ra0, dec0)
    #################################################################################
    if(len(finalObjIDstar)<1):
	return pixelNo, inde.size
    else:
        return finalObjIDstar, finalRaArrayStar, finalDecArrayStar, raErrStarInPixel, decErrStarInPixel, mjdStarInPixel,rMagStarInPixel, nObsStarInPixel

In [ ]:
def correctedStars(packParameterList, h5correctedStarsFile):
    # table definition
    class Star(tables.IsDescription):
	# in LSD, obj_id is a 64-bit unsigned integer,
	# but pytables cannot index 64-bit unsigned integers
	# so they are saved as 64-bit signed integers
	obj_id = tables.Int64Col(pos=0)
	ra  = tables.Float64Col(pos=1)
	dec = tables.Float64Col(pos=2)
	raErr = tables.Float64Col(pos=3)
	decErr = tables.Float64Col(pos=4)
	mjd  = tables.Float64Col(pos=5)
	mr = tables.Float64Col(pos=6)
	nObs = tables.Float64Col(pos=7)

    # open a pytable file
    h5file = tables.open_file(h5correctedStarsFile, mode = "w", title = "try1")
    # define compression filters
    filters = tables.Filters(complib='blosc', complevel=5)
    # create the table
    #create_table(where, name, obj, title, expectedrows, filters)
    #"/" refers to h5file.root object
    table = h5file.create_table('/', 'table1', Star, "tryTable", expectedrows=100563159, filters=filters)
    star = table.row
    #start workers
    pool = Pool(processes=20)
    ti = time()
    #chunk size is used to submit jobs in batches which reduces overhead
    iterator = pool.imap_unordered(pixelTasksCombinedData,packParameterList, chunksize=500)
    counterForRows = 0
    noOfPixelsIterated = 0
    dat  = []
    for res in iterator:
        if (len(res)==2):
	    print "there was an error with :( ", res
	else:
	    objID_, ra_, dec_, raErr_, decErr_, mjd_, mr_, nObs_ = res
	    noOfPixelsIterated += 1
	    print "noOfPixelsIterated",noOfPixelsIterated
            #so that you donot encounter empty pixels!
	    if (objID_.size != 0):
		for i in range(0, objID_.size):    
		    star['obj_id']  = (objID_[i]).astype('i8')
		    star['ra']  = (ra_[i]).astype('float')
		    star['dec']  = (dec_[i]).astype('float')
		    star['raErr']  = (raErr_[i]).astype('float')
		    star['decErr']  = (decErr_[i]).astype('float')
		    star['mjd']  = (mjd_[i]).astype('float')
		    star['mr']  = (mr_[i]).astype('float')
		    star['nObs']  = (nObs_[i]).astype('float')
		    star.append()
		    counterForRows += 1
        	if (counterForRows % 10000 == 0): 
            	    table.flush()
	    if (objID_.size ==0):
	 	print "zero array"

    #print "no of rows in table we began with", noOfRows
    #terminate the pool of multiprocessors
    pool.terminate()
    table.flush()
    noOfRowsInTable = table.nrows
    print "noOfRowsInTable:", noOfRowsInTable
    # create a full index on the obj_id column
    indexrows = table.cols.obj_id.create_csindex(filters=tables.Filters(complib='blosc', complevel=5))
    # create a new table that is sorted by obj_id
    sI = table.cols.obj_id
    table2 = tables.Table.copy(table, newname='correctedStar', overwrite=True, sortby=sI, checkCSI=True, propindexes=True)
    #  noOfRowsInTable2 = table2.nrows
    # delete the unsorted table
    h5file.root.table1.remove()    
    h5file.close()